In [192]:
import sys
import re
import pandas as pd
import tweepy
import psycopg2
from os import getenv
from dotenv import load_dotenv
load_dotenv()

True

In [193]:
ELEPHANTSQL_DATABASE = getenv('ELEPHANTSQL_DATABASE')
ELEPHANTSQL_USERNAME = getenv('ELEPHANTSQL_USERNAME')
ELEPHANTSQL_PASSWORD = getenv('ELEPHANTSQL_PASSWORD')
ELEPHANTSQL_HOST = getenv('ELEPHANTSQL_HOST')

TWITTER_CONSUMER_API_KEY = getenv('TWITTER_CONSUMER_API_KEY')
TWITTER_CONSUMER_API_SECRET = getenv('TWITTER_CONSUMER_API_SECRET')
TWITTER_ACCESS_TOKEN = getenv('TWITTER_ACCESS_TOKEN')
TWITTER_ACCESS_TOKEN_SECRET = getenv('TWITTER_ACCESS_TOKEN_SECRET')

TWITTER_AUTH = tweepy.OAuthHandler(TWITTER_CONSUMER_API_KEY,TWITTER_CONSUMER_API_SECRET)
TWITTER_AUTH.set_access_token(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
TWITTER = tweepy.API(TWITTER_AUTH)

In [194]:
def connect(DATABASE, USERNAME, PASSWORD, HOST):
    """ Connect to the PostgreSQL database server """
    elephantsql_client = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')

        # Connect to ElephantSQL-hosted PostgreSQL
        elephantsql_client = psycopg2.connect(dbname=DATABASE, user=USERNAME, password=PASSWORD, host=HOST)

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    return elephantsql_client

In [195]:
def user_tweets_table(elephantsql_client, user):
    '''Creating table for a users tweets'''

    # Initializing table for new user
    command = '''CREATE TABLE IF NOT EXISTS {}_tweets_table (tweet       varchar(500))'''.format(user[1:])
    
    try:

        # A "cursor", a structure to iterate over db records to perform queries
        cur = elephantsql_client.cursor()

        # Execute commands in order
        cur.execute(command)

        # Close communication with the PostgreSQL database server
        cur.close()

        # Commit the changes
        elephantsql_client.commit()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

In [196]:
def usernames_table(elephantsql_client):
    '''Creating table for a all usernames'''

    # Initializing table for new user
    command = '''
        CREATE TABLE IF NOT EXISTS usernames_table (username       varchar(30))                                    
        '''
    
    try:

        # A "cursor", a structure to iterate over db records to perform queries
        cur = elephantsql_client.cursor()

        # Execute commands in order
        cur.execute(command)

        # Close communication with the PostgreSQL database server
        cur.close()

        # Commit the changes
        elephantsql_client.commit()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)


In [197]:
def recent_table(elephantsql_client):
    '''Creating table for recent tweet comparisons'''

    # Initializing table for new user
    command = '''
        CREATE TABLE IF NOT EXISTS comparision_table (comparision_str       varchar(500))                                    
        '''
    
    try:

        # A "cursor", a structure to iterate over db records to perform queries
        cur = elephantsql_client.cursor()

        # Execute commands in order
        cur.execute(command)

        # Close communication with the PostgreSQL database server
        cur.close()

        # Commit the changes
        elephantsql_client.commit()

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)


In [198]:
def single_insert(elephantsql_client, insert_req):
    """ Execute a single INSERT request """
    cur = elephantsql_client.cursor()
    try:
        cur.execute(insert_req)
        elephantsql_client.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        elephantsql_client.rollback()
        cur.close()
        return 1
    cur.close()

In [199]:
def execute_many(elephantsql_client, df, table):
    """
    Using cursor.executemany() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]

    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s)" % (table, 'tweet')
    cursor = elephantsql_client.cursor()
    try:
        cursor.executemany(query, tuples)
        elephantsql_client.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        elephantsql_client.rollback()
        cursor.close()
        return 1
    print("execute_many() done")
    cursor.close()

In [200]:
def clean_tweet(tweet):
    '''Cleaning raw tweet for modeling'''

    tweet = tweet.lower()
    emoji_list = tweet.split()                  # Creating list to reference emojis
    tweet = re.sub('[^a-z 0-9]', '', tweet)
    tweet = tweet.split(' ')
    output_string = ''

    # Creating output string, handeling links and emojis
    for index, token in enumerate(tweet):
        if token[0:4] == 'http':
            pass
        elif token == '':
            try:
                output_string = output_string + ' ' + str(ord(emoji_list[index]))   # Replacing emoji with number value
            except:
                # Token is not an emoji
                pass
        else:
            output_string = output_string + ' ' + token

    output_string = output_string[1:]

    return output_string

In [201]:
def generate_tweets(user):
    tweets = []

    for status in tweepy.Cursor(TWITTER.user_timeline, screen_name=user, tweet_mode="extended", count=200).items():
        tweets.append(clean_tweet(status.full_text))

    return pd.DataFrame({'tweet':tweets})

**Initial Data**

In [202]:
# Initial users
user1 = '@nasa'
user2 = '@barackobama'
user3 = '@justinbieber'
user4 = '@billgates'

# Recent Comparions
comp1 = '@nasa is more likely to say "This week at NASA, we celebrate @NASAPersevere’s first landiversary, admire new X-ray images from #IXPE, and prepare for the next @NorthropGrumman resupply mission to the @SpaceStation." than @justinbieber'

**Create tweets tables**

In [203]:
# Database connection
elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

# Building Inital user tweet tables
user_tweets_table(elephantsql_client, user1)
user_tweets_table(elephantsql_client, user2)
user_tweets_table(elephantsql_client, user3)
user_tweets_table(elephantsql_client, user4)

# Generating dataframes
df1 = generate_tweets(user1)
df2 = generate_tweets(user2)
df3 = generate_tweets(user3)
df4 = generate_tweets(user4)

# Inserting data from dataframes
execute_many(elephantsql_client, df1, '{}_tweets_table'.format(user1[1:]))
execute_many(elephantsql_client, df2, '{}_tweets_table'.format(user2[1:]))
execute_many(elephantsql_client, df3, '{}_tweets_table'.format(user3[1:]))
execute_many(elephantsql_client, df4, '{}_tweets_table'.format(user4[1:]))

# Close the connection
elephantsql_client.close()
print('Connection is closed.')

Connecting to the PostgreSQL database...
execute_many() done
execute_many() done
execute_many() done
execute_many() done
Connection is closed.


**Create usernames table**

In [204]:
# Database connection
elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

# Initializing usernames table
usernames_table(elephantsql_client)

# Adding users to the usernames table
query = "INSERT INTO usernames_table (username) VALUES ('{}')".format(user1)
single_insert(elephantsql_client, query)

query = "INSERT INTO usernames_table (username) VALUES ('{}')".format(user2)
single_insert(elephantsql_client, query)

query = "INSERT INTO usernames_table (username) VALUES ('{}')".format(user3)
single_insert(elephantsql_client, query)

query = "INSERT INTO usernames_table (username) VALUES ('{}')".format(user4)
single_insert(elephantsql_client, query)

# Close the connection
elephantsql_client.close()
print('Connection is closed.')

Connecting to the PostgreSQL database...
Connection is closed.


**Create comparisions table**

In [205]:
# Database connection
elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

# Initializing recent comparisions table
recent_table(elephantsql_client)

# Adding recent comparisions
query = "INSERT INTO comparision_table (comparision_str) VALUES ('{}')".format(comp1)
single_insert(elephantsql_client, query)

# Close the connection
elephantsql_client.close()
print('Connection is closed.')

Connecting to the PostgreSQL database...
Connection is closed.


**Retrive list of tweets for given user**

In [206]:
def get_user_tweets(user):
    # Creating the connection to database
    elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

    # A "cursor", a structure to iterate over db records to perform queries
    cur = elephantsql_client.cursor()

    command = '''
    SELECT tweet FROM {}_tweets_table
    '''.format(user[1:])

    # Execute commands in order
    cur.execute(command)

    tweets = []
    tweets_list = cur.fetchall()
    for tup in tweets_list:
        tweets.append(tup[0])

    # Close communication with the PostgreSQL database server
    cur.close()

    # Commit the changes
    elephantsql_client.commit()

    # Close the connection
    elephantsql_client.close()
    print('Connection is closed.')

    return tweets

**Get Users**

In [207]:
def get_users():
    # Creating the connection to database
    elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

    # A "cursor", a structure to iterate over db records to perform queries
    cur = elephantsql_client.cursor()

    command = '''
    SELECT username FROM usernames_table
    '''

    # Execute commands in order
    cur.execute(command)

    users = []
    users_list = cur.fetchall()
    for tup in users_list:
        users.append(tup[0])

    # Close communication with the PostgreSQL database server
    cur.close()

    # Commit the changes
    elephantsql_client.commit()

    # Close the connection
    elephantsql_client.close()
    print('Connection is closed.')

    return users

**Get comparisons**

In [208]:
def get_comparisions():
    # Creating the connection to database
    elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

    # A "cursor", a structure to iterate over db records to perform queries
    cur = elephantsql_client.cursor()

    command = '''
    SELECT comparision_str FROM comparision_table
    '''

    # Execute commands in order
    cur.execute(command)

    comparisions = []
    comparisions_list = cur.fetchall()
    for tup in comparisions_list:
        comparisions.append(tup[0])

    # Close communication with the PostgreSQL database server
    cur.close()

    # Commit the changes
    elephantsql_client.commit()

    # Close the connection
    elephantsql_client.close()
    print('Connection is closed.')

    return comparisions

In [209]:
user1_tweets = get_user_tweets(user1)
usernames = get_users()
comparisions = get_comparisions()

Connecting to the PostgreSQL database...
Connection is closed.
Connecting to the PostgreSQL database...
Connection is closed.
Connecting to the PostgreSQL database...
Connection is closed.


**Delete all tables**

In [210]:
def drop_all_tables():
    # Creating the connection to database
    elephantsql_client = connect(ELEPHANTSQL_DATABASE, ELEPHANTSQL_USERNAME, ELEPHANTSQL_PASSWORD, ELEPHANTSQL_HOST)

    # A "cursor", a structure to iterate over db records to perform queries
    cur = elephantsql_client.cursor()

    users = get_users()
    
    for user in users:
        command = '''
                DROP TABLE {}_tweets_table;
                '''.format(user[1:])
        # Execute commands in order
        cur.execute(command)

    command = '''
    DROP TABLE usernames_table;
    '''
    cur.execute(command)

    command = '''
    DROP TABLE comparision_table;
    '''
    cur.execute(command)

    # Close communication with the PostgreSQL database server
    cur.close()

    # Commit the changes
    elephantsql_client.commit()

    # Close the connection
    elephantsql_client.close()
    print('Connection is closed.')

    return comparisions